Tarquin Bennett                

# Question 1 

### 3.10 
cost of sub is 2

`
/ # b r i e f
#[0,1,2,3,4,5]
d[1,1,2,3,4,5]
r[2,2,1,2,3,4]
i[3,3,2,1,2,3]
v[4,4,3,2,2,3]
e[5,5,4,3,2,3]
`

`
 /# d r i v e r s
#[0,1,2,3,4,5,6,7]
d[1,0,1,2,3,4,5,6]
r[2,1,0,1,2,3,4,5]
i[3,2,1,0,1,2,3,4]
v[4,3,2,1,0,1,2,3]
e[5,4,3,2,1,0,1,2]
`

Edit distance for brief to drive is 3 while drive to drivers is 2. Which means drive is closer in edit distance to drivers.

### 3.11

In [6]:
def editDistance(n: str,m: str) -> int:
    lenN=len(n)
    lenM=len(m)

    D=[[0 for _ in range(lenM+1)] for _ in range(lenN+1)]

    for i in range(1,lenN+1):
        D[i][0] = D[i-1][0]+1

    for i in range(1,lenM+1):
        D[0][i] = D[0][i-1]+1

    for i in range(1,lenN+1):
        for j in range(1,lenM+1):
            if n[i-1]==m[j-1]:
                D[i][j]=D[i-1][j-1]
            else:
                D[i][j] = min(D[i-1][j]+1,D[i-1][j-1]+1,D[i][j-1]+1)
    #prints the grid
    for i in D:
        print(i)
    return D[lenN][lenM]

In [7]:
editDistance("drive","brief")

[0, 1, 2, 3, 4, 5]
[1, 1, 2, 3, 4, 5]
[2, 2, 1, 2, 3, 4]
[3, 3, 2, 1, 2, 3]
[4, 4, 3, 2, 2, 3]
[5, 5, 4, 3, 2, 3]


3

In [9]:
editDistance("drive","drivers")

[0, 1, 2, 3, 4, 5, 6, 7]
[1, 0, 1, 2, 3, 4, 5, 6]
[2, 1, 0, 1, 2, 3, 4, 5]
[3, 2, 1, 0, 1, 2, 3, 4]
[4, 3, 2, 1, 0, 1, 2, 3]
[5, 4, 3, 2, 1, 0, 1, 2]


2

# Question 2

In [1]:
import nltk
from nltk.corpus import movie_reviews
import random

documents = [(list(movie_reviews.words(fileid)), category)
            for category in movie_reviews.categories()
            for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)

all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

featuresets = [(document_features(d),c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(5)

0.84
Most Informative Features
        contains(seagal) = True              neg : pos    =     11.0 : 1.0
   contains(outstanding) = True              pos : neg    =     11.0 : 1.0
         contains(mulan) = True              pos : neg    =      8.3 : 1.0
   contains(wonderfully) = True              pos : neg    =      7.6 : 1.0
         contains(damon) = True              pos : neg    =      6.1 : 1.0


This shows us the model we trained has a 87% accuracy in correctly classifying positive or negative reviews. We also see the top 5 words that tell us the most information on whether the review is positive or negative. So this means that if we take this model and apply it to undecided data set, there is 87% accuracy that the model will be correct in prediction of positive or negative.

# Question 3 

As seen in "Reviews Classification Using SentiWordNet Lexicon", when using SentiWordNet you can classify reviews as positive and negative. This research paper talks about a previous method term counting, and compares it to two new methods that the researchers came up with, sum on review and average on sentence and Average on Review. Term counting is when each word is assigned a positive class or negative class and then counted up and the review is classified base on which class has the higher count. In their research it was accurate 56.77% of the time. Sum on review is when the summation of positive and negative scores for each term is found in a review, is calculated to get the positive and negative scores for all review words. Then based on which score is higher, the review is given that sentiment. This is different from term counting because it takes into account the magnitude scores for the words. This research saw the accuracy of 67% for this method. Average on sentence and Average on Review is when the average sentiment is found for each sentence and then the average sentiment is found for all sentence to get the review its sentiment. The accuracy for this method is 68.63%. Therefore using SentiWordNet with the average on sentence and average on review method, along with Python and WordNet, you can assign a sentiment for a review with a 68.63% accuracy, which is about 10% higher then term counting. 

# Question 4 

In [32]:
from nltk.corpus import senseval
instances = senseval.instances('hard.pos')
size = int(len(instances) * 0.1)

def features(instance):
    feat = dict()
    p = instance.position
       ## previous word and tag
    if p: ## > 0
        feat['wp'] = instance.context[p-1][0]
        feat['tp'] = instance.context[p-1][1]
       ## use BOS if it is the first word
    else: # 
        feat['wp'] = (p, 'BOS')
        feat['tp'] = (p, 'BOS')
       ## following word and tag       
        feat['wf'] = instance.context[p+1][0]
        feat['tf'] = instance.context[p+1][1]
    return feat


featureset =[(features(i), i.senses[0]) for i in 
             instances if len(i.senses)==1]

random.shuffle(featureset)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print ("Accuracy on Test:", nltk.classify.accuracy(classifier, test_set))

Accuracy on Test: 0.8267898383371824


This naive bayes classifier predicts the correct sense tag for a given instance with an accuracy of 82.68%. This allows us to be confidante with the model to apply it to a larger data set and have us get the current sense tag. To improve the accuracy, we would need to train the model on a bigger dataset.